# 🤺 Demo: Using the Bedrock Agents

**What is a Bedrock Agent?**

Bedrock, and more specifically its Converse API, has tool calling built-in. As a result, it made sense to overhaul the Bedrock LLM classes to support said tool calling (which involved extra models, handling the output, etc.).

Once the model classes had been rebuilt, the opportunity presented itself to implement a new Agent. Rather than build a vanilla ReAct agent, this one would leverage implicit tool calling, and support some extra features. Let's start off slow.

**_Caveat_** Agents work with various models, but they need be 'smart enough'. For instance, NovaPro seems to work quite well, whereas NovaLite of NovaMicro tend to make mistakes in the tool calling, or returning of answers.

### 1. A simple Bedrock Agent

In [9]:
from fence.agents.bedrock import BedrockAgent
from fence.models.bedrock import NovaPro
from fence.models.bedrock import Claude35Sonnet, ClaudeHaiku

In [10]:
from fence.utils.logger import setup_logging
setup_logging(log_level="DEBUG")
# Create a model
model = NovaPro(region='us-east-1') # Not out in eu-west-1 yet, our default

# Create an agent
agent = BedrockAgent(model=model)

# Run the agent
response = agent.run('hello!')

print(f"\n\nAgent answer: {response.answer}")

2025-04-25 11:25:38,627 [DEBUG] [fence.models.bedrock.base.__init__:204] Initialized Bedrock client in region: us-east-1
2025-04-25 11:25:38,627 [DEBUG] [fence.agents.bedrock.agent._build_system_message:215] System message: 
You are a helpful assistant. You can think in <thinking> tags, and provide an answer in <answer> tags. Try to always plan your next steps in <thinking> tags. Make sure to exhaust all your tools and delegate capabilities before providing your final answer.

2025-04-25 11:25:38,628 [DEBUG] [fence.agents.bedrock.agent._register_tools:536] Agent BedrockAgent: Cleared toolConfig (no tools available)
2025-04-25 11:25:38,628 [DEBUG] [fence.agents.bedrock.agent._invoke_iteration:693] Agent BedrockAgent is invoking the model:
	[toolConfig]:
None
	[prompt]:
{   'messages': [{'content': 'hello!', 'role': 'user'}],
    'system': '\n'
              'You are a helpful assistant. You can think in <thinking> tags, '
              'and provide an answer in <answer> tags. Try to alw

Cool, that's a hello world agent. Let's make it a bit more interesting and give it some more tools.

### 2. A Bedrock Agent that can perform temperature conversions


In [11]:
from fence.tools.base import tool

# Define a weather tool using the decorator
@tool(description="Get the current weather for a location")
def get_weather(location: str):
    """
    Get the current weather for a location.

    :param location: The location to get the weather for
    :return: The current weather for the location
    """
    # Simulate weather API response
    weather_data = {
        "New York": "Sunny, 75°F",
        "London": "Rainy, 55°F",
        "Tokyo": "Cloudy, 65°F",
        "Sydney": "Clear, 80°F",
    }
    return weather_data.get(location, f"Weather data not available for {location}")

# Define a temperature conversion tool
@tool(description="Convert temperature between Fahrenheit and Celsius")
def convert_temperature(value: float, from_unit: str, to_unit: str):
    """
    Convert temperature between Fahrenheit and Celsius.

    :param value: The temperature value to convert
    :param from_unit: The unit to convert from ('Fahrenheit', 'F', 'Celsius', 'C')
    :param to_unit: The unit to convert to ('Fahrenheit', 'F', 'Celsius', 'C')
    :return: The converted temperature value
    """
    # Normalize input units
    from_unit = from_unit.lower()
    to_unit = to_unit.lower()

    # Convert full names to single letters
    if from_unit in ["fahrenheit", "f"]:
        from_unit = "f"
    elif from_unit in ["celsius", "c"]:
        from_unit = "c"

    if to_unit in ["fahrenheit", "f"]:
        to_unit = "f"
    elif to_unit in ["celsius", "c"]:
        to_unit = "c"

    if from_unit == "f" and to_unit == "c":
        return f"{(value - 32) * 5/9:.1f}°C"
    elif from_unit == "c" and to_unit == "f":
        return f"{(value * 9/5) + 32:.1f}°F"
    else:
        return f"Invalid conversion: {from_unit} to {to_unit}"


Now let's hand these over to the agent, and see how it can use them.

In [12]:
# New agent
agent = BedrockAgent(model=model, tools=[get_weather, convert_temperature])

# Get the weather in Brussels, in Celsius
response = agent.run('What is the weather in New York, in Celsius?')

print(f"\n\nAgent answer: {response.answer}\n\n")


2025-04-25 11:25:39,881 [DEBUG] [fence.agents.bedrock.agent._build_system_message:215] System message: 
You are a helpful assistant. You can think in <thinking> tags, and provide an answer in <answer> tags. Try to always plan your next steps in <thinking> tags. Make sure to exhaust all your tools and delegate capabilities before providing your final answer.

2025-04-25 11:25:39,881 [DEBUG] [fence.agents.bedrock.agent._register_tools:554] Agent BedrockAgent: Registered 2 tools with Bedrock model: ['GetWeatherTool', 'ConvertTemperatureTool']
2025-04-25 11:25:39,882 [DEBUG] [fence.agents.bedrock.agent._invoke_iteration:693] Agent BedrockAgent is invoking the model:
	[toolConfig]:
BedrockToolConfig(tools=[BedrockTool(toolSpec=BedrockToolSpec(name='GetWeatherTool', description='Get the current weather for a location', inputSchema=BedrockToolInputSchema(json_field=BedrockJSONSchema(type='object', properties={'location': {'type': 'string', 'description': 'Parameter location for the GetWeather

Now, the response actually contains more information than just the answer. What it contains is the following:

- `answer`: The answer to the question
- `events`: All events that occurred during the agent's execution, broken down into:
  - `thinking`: All thoughts the agent had
  - `tool_use`: All tool calls the agent made
  - `answer`: The final answer to the question

This allows for a much more detailed analysis of the agent's thought process and tool usage.


In [13]:
from pprint import pprint
pprint(response.events)

[Thinking event: "To find out the weather in New York in Celsius, I need to follow these steps:
1. Get the current weather in New York using the GetWeatherTool.
2. Convert the temperature from Fahrenheit to Celsius using the ConvertTemperatureTool.",
 Tool_use event: GetWeatherTool ({'location': 'New York'}) -> Sunny, 75°F,
 Thinking event: "The current weather in New York is sunny with a temperature of 75°F. I need to convert this temperature from Fahrenheit to Celsius using the ConvertTemperatureTool.",
 Tool_use event: ConvertTemperatureTool ({'from_unit': 'Fahrenheit', 'to_unit': 'Celsius', 'value': 75}) -> 23.9°C,
 Answer event: "The weather in New York is sunny with a temperature of 23.9°C."]


Some extra options:
- Pass an `identifier` to the agent, to help with logging (useful when running multiple agents)
- Pass a `system_message` to the agent, to change the assistant's personality
- Pass a `memory` to the agent, to use a custom memory (e.g., DynamoDBMemory)
- Pass `log_agentic_response` to the agent, to disable intermediate logging
- Pass a `are_you_serious` to the agent, to change the log message style to boring



In [14]:
print("-- Using an identifier")
agent = BedrockAgent(model=model, identifier="my_agent")
agent.run('hello!')

print("-- Using a system message")
agent = BedrockAgent(model=model, system_message="You are a rude French person who fakes not knowing English.")
agent.run('hello!')

print("-- Boring logging")
agent = BedrockAgent(model=model, are_you_serious=True)
agent.run('hello!');


-- Using an identifier
2025-04-25 11:25:42,734 [DEBUG] [fence.agents.bedrock.agent._build_system_message:215] System message: 
You are a helpful assistant. You can think in <thinking> tags, and provide an answer in <answer> tags. Try to always plan your next steps in <thinking> tags. Make sure to exhaust all your tools and delegate capabilities before providing your final answer.

2025-04-25 11:25:42,734 [DEBUG] [fence.agents.bedrock.agent._register_tools:536] Agent my_agent: Cleared toolConfig (no tools available)
2025-04-25 11:25:42,735 [DEBUG] [fence.agents.bedrock.agent._invoke_iteration:693] Agent my_agent is invoking the model:
	[toolConfig]:
None
	[prompt]:
{   'messages': [{'content': 'hello!', 'role': 'user'}],
    'system': '\n'
              'You are a helpful assistant. You can think in <thinking> tags, '
              'and provide an answer in <answer> tags. Try to always plan your '
              'next steps in <thinking> tags. Make sure to exhaust all your '
            

Now let's get into some more interesting stuff.

### 3. Assigning custom event handlers

Bedrock agents, as you already saw, handle output as different styles of events. There can be handled separately. One such example you can already see in the agentic logs: depending on the event, the log message is formatted differently. However, this also opens the door for new opportunities. What if we want to send messages to a Slack channel when a tool is used? Or when the agent provides an answer? Or when it thinks? We can do this by assigning custom event handlers. Let's mock some with a simple example.



In [16]:
# Create a custom event handler
from fence.utils.logger import setup_logging
setup_logging()
import random

def on_tool_use(tool_name, parameters, result):
    """Handle tool use events."""
    print(f"SENDING TOOL USE TO SLACK: CALLED A TOOL: {tool_name} with {parameters} -> {result}")

def on_thinking(text):
    """Handle agent thinking events."""
    synonyms_for_thinking = ["thinking", "pondering", "considering", "evaluating", "analyzing", "reflecting", "considering", "evaluating", "analyzing", "reflecting", "considering", "evaluating", "analyzing", "reflecting"]
    print(f"SENDING THINKING TO SLACK: *{random.choice(synonyms_for_thinking)}*")

def on_answer(text):
    """Handle agent answer events."""
    print(f"SENDING ANSWER TO SLACK: {text}")


# Create the agent
agent = BedrockAgent(model=model, event_handlers={
    'on_tool_use': on_tool_use,
    'on_thinking': on_thinking,
    'on_answer': on_answer
},
 log_agentic_response=False,
 tools=[get_weather, convert_temperature])

# Run the agent
agent.run('What is the weather in New York, in Celsius?');


2025-04-25 11:27:33,964 [WARNING] [fence.agents.bedrock.agent._set_event_handlers:277] Converting dict to EventHandlers: {'on_tool_use': <function on_tool_use at 0x12106a160>, 'on_thinking': <function on_thinking at 0x1252418a0>, 'on_answer': <function on_answer at 0x1252409a0>}
SENDING THINKING TO SLACK: *reflecting*
SENDING TOOL USE TO SLACK: CALLED A TOOL: GetWeatherTool with {'location': 'New York'} -> Sunny, 75°F
SENDING THINKING TO SLACK: *considering*
SENDING TOOL USE TO SLACK: CALLED A TOOL: ConvertTemperatureTool with {'from_unit': 'Fahrenheit', 'to_unit': 'Celsius', 'value': 75} -> 23.9°C
SENDING ANSWER TO SLACK: The weather in New York is sunny with a temperature of 23.9°C.


In [23]:
from fence.tools.base import BaseTool
class MyIdentityTool(BaseTool):
    """
    Tool to get the contact information of the user interacting; Getting the identity of the user
    """
    def __init__(self):
        super().__init__()

    def execute_tool(self, **kwargs):
        """
        Execute the tool to get the identity / contact information of the user
        :return: Returns the identity / contact information of the user
        """
        
        return f"""name: Frank
user_id: 123
email: frank@example.com
        """

identity_tool = MyIdentityTool()
print(identity_tool.get_tool_name())


MyIdentityTool
